# Pipeline output creator

this script creates a nice output for the selected locus.
The output will have 8 sequence files, one png image, one metadata information file, one locusinfo text file, one readme

In [2]:
# First created 18/10/2019
# Made by L-F-S
# At the University Of Trento, Italy

from os import system, chdir
import sys
from numpy import unique
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna, generic_protein, RNAAlphabet
from Bio import pairwise2
sys.path.insert(0, '/home/lorenzo.signorini/cas_mining/utils/')
import filename_discrepancies

# parameters:
datadir="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/"


In [3]:
seqid="CPMHNJAN_01529" #name of sequence #sys.argv
outputdir=datadir+"8outputv1/"+seqid+"/"

# import output parameters
input_for_output=open(outputdir+seqid+"inputforoutput","r")
inputs=input_for_output.readlines()
matched_rep=Seq(inputs[0].strip())
rpt=Seq(inputs[1].strip())
matched_rep_start=int(inputs[2].strip())
matched_rep_stop=int(inputs[3].strip())
strand=int(inputs[4].strip())
terminator_end=int(inputs[5].strip())
terminator=Seq(inputs[6].strip())
feature=inputs[7].strip()
feature_strand=inputs[8].strip()
input_for_output.close()
print("strand",strand,"matched_rep_start",matched_rep_start,"matched_rep_stop", matched_rep_stop,"terminator_end",terminator_end)

FileNotFoundError: [Errno 2] No such file or directory: '/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/8outputv1/CPMHNJAN_01529/CPMHNJAN_01529inputforoutput'

In [92]:
# Import data
cas_dataset=pd.read_csv("/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/5caslocitable/known_"+feature+"prokkacas9_variants_table.csv", index_col=0)
cas_dataset.columns

Index(['Seq ID', 'Seq Description', 'Seq', 'Contig', 'Genome Name', 'Study',
       'Sample Name', 'SGB ID', 'Pos', 'pilercr_CRISPR', 'minced_CRISPR',
       'prokka_cas1', 'prokka_cas2', 'uSGB', 'Level of estimated taxonomy',
       'Estimated taxonomy', 'prokka_cas9'],
      dtype='object')

In [93]:
#perso la battaglia..
tabellazza=pd.read_csv(datadir+"/3tabellazza/crisprcas_hits_table.csv", index_col=0)


In [126]:
cas9_aa=cas_dataset[cas_dataset["Seq ID"]==seqid]["Seq"].iloc[0]
dataset=cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0]
genomename=cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0]
SGB=cas_dataset[cas_dataset["Seq ID"]==seqid]["SGB ID"].iloc[0]
uSGB=cas_dataset[cas_dataset["Seq ID"]==seqid]["uSGB"].iloc[0]
est_tax=cas_dataset[cas_dataset["Seq ID"]==seqid]["Estimated taxonomy"].iloc[0]
l_est_tax=cas_dataset[cas_dataset["Seq ID"]==seqid]["Level of estimated taxonomy"].iloc[0]
contigname=cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0]

In [127]:
position_of={}

unostranosplit=cas_dataset[cas_dataset["Seq ID"]==seqid]["minced_CRISPR"].iloc[0].split("\', \'")
position_of["CRISPR"]= (int(unostranosplit[1]), int(unostranosplit[2].strip("\')]\"")))


print("\n-----------------------------------------------------\n"+feature+" id:\t"+seqid+"\nSGB:\t"+str(SGB)+"\nGenome Name:\t",\
      genomename+"\nContig:\t"+cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0],"\n-----------------------------------------------------\n")

dataset, genomename, samplename, s3_dataset, s3_genomename = filename_discrepancies.master_filename_discrepancies(dataset,genomename)

# persa la battagliaaaaa
prokka_anno=tabellazza[tabellazza["Genome Name"]==s3_genomename]["prokka_cas"].iloc[0]
# so già che all casses inside here are on the same contig which is 'contig', xke le ho estratte per essere così

prokkaslist=prokka_anno.split(">")
prokkaslist.pop(0)
# get contigname of current effectorcas   
cas_position={}
for casanno in prokkaslist:
    unostranosplit=casanno.split("Name=")
    casnumber=unostranosplit[1][3] #because we know that we are loooking at 'good' loci, this is just going to be 9, 1 or 2
    unaltrostranosplit=casanno.split("\t")
    cas_start=unaltrostranosplit[3]
    cas_stop=unaltrostranosplit[4]
    cas_position["Cas"+casnumber]=(int(cas_start),int(cas_stop))
position_of["Cas9"]=cas_position["Cas9"]
#persalabattagliaaa
 

# access contig
genomefilename= "/shares/CIBIO-Storage/CM/scratch/tmp_projects/epasolli_darkmatter/allcontigs/ALLreconstructedgenomes/"+str(SGB)+"/"+genomename+".fa"
for record in SeqIO.parse(genomefilename,"fasta"):  #c'è modo di non fare questo 'ciclo'?
    if record.id.startswith(contigname):
        #test se effettivamente la seq di cas è dove voglio io

        SeqIO.write(record, "/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/tempblastdb", "fasta")
        Cas2info=cas_dataset[cas_dataset["Seq ID"]==seqid]["prokka_cas2"].iloc[0]
        Cas2ID=Cas2info.lstrip("[(\'ID=").split("\', ")[0]
        Cas1info=cas_dataset[cas_dataset["Seq ID"]==seqid]["prokka_cas1"].iloc[0]
        Cas1ID=Cas1info.lstrip("[(\'ID=").split("\', ")[0]
    #    Cas9info=cas_dataset[cas_dataset["Seq ID"]==seqid]["prokka_cas9"].iloc[0]
     #   print(Cas9info)
        position_of["Cas1"]=(int(Cas1info.split("\', \'")[1]), int(Cas1info.split("\', \'")[2].rstrip("\')]")))
        position_of["Cas2"]=(int(Cas2info.split("\', \'")[1]), int(Cas2info.split("\', \'")[2].rstrip("\')]")))
        position_of["Cas9"]=cas_position["Cas9"]
  #      position_of["Cas9"]=(int(Cas9info.split("\', \'")[1]), int(Cas9info.split("\', \'")[2].rstrip("\')]")))
        print("CRISPR si trova", position_of["CRISPR"])
        print("Cas9 si trova", position_of["Cas9"])
        print("Cas2 si trova", Cas2info)
        print("Cas1 si trova", Cas1info)

       # mettici l'estrattore di Cas9 nt..
    #poi devi aggeggiare anche il minced per avere solo il contig che vuoi..


-----------------------------------------------------
Cas9 id:	CJFLGLGO_01367
SGB:	4329
Genome Name:	 Obregon-TitoAJ_2015__SM40__bin.4
Contig:	NODE_137_length_63930_cov_6.02239 
-----------------------------------------------------

CRISPR si trova (44486, 44718)
Cas9 si trova (46031, 49270)
Cas2 si trova [('ID=CJFLGLGO_01365', '44816', '45136')]
Cas1 si trova [('ID=CJFLGLGO_01366', '45129', '46028')]


In [128]:
print(position_of, cas_position) #persalabattagliaa

{'CRISPR': (44486, 44718), 'Cas9': (46031, 49270), 'Cas1': (45129, 46028), 'Cas2': (44816, 45136)} {'Cas2': (44816, 45136), 'Cas1': (45129, 46028), 'Cas9': (46031, 49270)}


In [129]:
min_locus=min([value[0] for value in position_of.values()])
max_locus=max([value[1] for value in position_of.values()])
print(min_locus, max_locus)

44486 49270


In [130]:
for record in SeqIO.parse("/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/selected_loci/"+seqid+"/"+genomename+".fa", "fasta"):
    if record.id==contigname:
        contigseq=record.seq
if strand<0:
    tracrRNA=contigseq[terminator_end:matched_rep_stop].reverse_complement()
    position_of["tracrRNA"]=(terminator_end, matched_rep_stop)

else:
    tracrRNA=contigseq[matched_rep_start:terminator_end]
    position_of["tracrRNA"]=(matched_rep_start,terminator_end)

    
tracrRNA=tracrRNA.transcribe()
print("tracrRNA:", tracrRNA, len(tracrRNA))


tracrRNA: UUAUAACAUACCAAGAACAAUUGGGUUACUACAAAAAGGUAGAAAACCGAAAAGCUCUGACGGCUCCUUUUUGGAGCCGUUAUCUUUUU 89


In [131]:
#v1.1 add complete locus
whole_locus=contigseq[min_locus-1:max_locus]


## Save output

In [132]:
#save output to folder:
system("mkdir "+outputdir)
coso_non_capisco_piu_nulla={}
# get sequences of cas9, cas1 e cas2 (in futuro le aggiungo direttamente al 5caslocus dataset, cosi ho tt li), both in aa and nt.
for cosa in [".faa", ".ffn"]:
    coso_non_capisco_piu_nulla[cosa]={}
    print("writing "+cosa+" files for cas proteins...")
    prokka_anno_file="/scratchCM/tmp_projects/epasolli_darkmatter/allcontigs/"+dataset+"/metabat/genomes_comp50_cont05/prokka/"+genomename+"/"+genomename+cosa
    for record in SeqIO.parse(prokka_anno_file,"fasta"):
        if record.id.startswith(seqid):
            record.description="Cas9 len="+str(len(record.seq))+" genome="+s3_genomename+" SGB="+str(SGB)+" contig="+contigname
            Cas9_fasta_header=">"+seqid+" "+record.description
            coso_non_capisco_piu_nulla[cosa]["Cas9"]=record.seq
            SeqIO.write(record, outputdir+"Cas9_"+seqid+cosa,"fasta")
        if record.id.startswith(Cas2ID):
            record.description="Cas2 len="+str(len(record.seq))+" genome="+s3_genomename+" SGB="+str(SGB)+" contig="+contigname
            Cas2_fasta_header=">"+Cas2ID+" "+record.description
            SeqIO.write(record, outputdir+"Cas2_"+seqid+cosa,"fasta")
            coso_non_capisco_piu_nulla[cosa]["Cas2"]=record.seq
        if record.id.startswith(Cas1ID):
            record.description="Cas1 len="+str(len(record.seq))+" genome="+s3_genomename+" SGB="+str(SGB)+" contig="+contigname
            Cas1_fasta_header=">"+Cas1ID+" "+record.description
            coso_non_capisco_piu_nulla[cosa]["Cas1"]=record.seq
            SeqIO.write(record, outputdir+"Cas1_"+seqid+cosa,"fasta")

# copy minced CRISPR array file:
print("copying minced CRISPR array...")
mincedCRISPRfilename=datadir+"1crisprsearch/out/"+s3_dataset+"/"+s3_genomename+".fa.minced.out"
system("cp "+mincedCRISPRfilename+" "+outputdir+"CRISPRarray_minced_"+seqid+".txt")
        

#autro
print("writing tracrRNA fasta file...")
f=open(outputdir+"tracr_"+seqid+".ffn","w")
f.write(">"+seqid+ " tracrRNA len="+str(len(tracrRNA))+" genome="+s3_genomename+" SGB="+str(SGB)+" contig="+contigname+"\n"+str(tracrRNA))
f.close()
#whole locus v1.1
print("writing whole locus fasta file...")
f=open(outputdir+"whole_locus_"+seqid+".ffn","w")
f.write(">"+seqid+ " CRISPR/Cas_complete_locus len="+str(len(whole_locus))+" genome="+s3_genomename+" SGB="+str(SGB)+" contig="+contigname+"\n"+str(whole_locus))
f.close()
print("Done.")

writing .faa files for cas proteins...
writing .ffn files for cas proteins...
copying minced CRISPR array...
writing tracrRNA fasta file...
writing whole locus fasta file...
Done.


In [133]:
#write Locus info text file
from datetime import date
import textwrap
today=date.today().strftime("%d/%m/%Y")

In [134]:
header="% created on "+today+"\n% author: L-F-S\n% affiliation: University of Trento, Italy\n% e-mail: lorenzo.signorini@alumni.unitn.it\n% source code available at: https://github.com/L-F-S/cas_mining\n\n"
title="\n"+"="*80+"\n\n"+" "*int((80-len(seqid+" LOCUS"))/2)+seqid+" LOCUS"+" "*int((80-len(seqid+" LOCUS"))/2)+"\n\n"+"="*80


In [135]:
Cas_header="\n\n\n\n\t\t\t\t+-------------+\n\t\t\t\t Cas Proteins\n\t\t\t\t+-------------+\n"
tracr_header="\n\n\n\t\t\t\t+-------------+\n\t\t\t\t   tracrRNA \n\t\t\t\t+-------------+\n"
crispr_header="\n\n\t\t\t\t+-------------+\n\t\t\t\t  CRISPRarray \n\t\t\t\t+-------------+\n"
generatedwithminced="generated with Minced\n\n\tCharles Bland et al., CRISPR Recognition Tool (CRT): a tool for automatic\n\tdetection of clustered regularly interspaced palindromic repeats, BMC\n\tBioinformatics 8, no. 1 (2007): 209.\n\n"



In [136]:
#v1.1
cas1_out="  | Cas1 |        \tprokka ID:"+Cas1ID+"\n\tstart relative to contig: "+str(position_of["Cas1"][0])+"\t| start relative to locus: "+str(position_of["Cas1"][0]-min_locus)+" \n\t stop relative to contig: "+str(position_of["Cas1"][1])+"\t|  stop relative to locus: "+str(position_of["Cas1"][1]-min_locus)+" \n\t  length (nt): "+str(position_of["Cas1"][1]-position_of["Cas1"][0])+" \n\t  length (aa): "+str(int((position_of["Cas1"][1]-position_of["Cas1"][0])/3))+"\n"
cas2_out="  | Cas2 |        \tprokka ID:"+Cas2ID+"\n\tstart relative to contig: "+str(position_of["Cas2"][0])+"\t| start relative to locus: "+str(position_of["Cas2"][0]-min_locus)+" \n\t stop relative to contig: "+str(position_of["Cas2"][1])+"\t|  stop relative to locus: "+str(position_of["Cas2"][1]-min_locus)+" \n\t  length (nt): "+str(position_of["Cas2"][1]-position_of["Cas2"][0])+" \n\t  length (aa): "+str(int((position_of["Cas2"][1]-position_of["Cas2"][0])/3))+"\n"
cas9_out="  | Cas9 |        \tprokka ID:"+seqid+"\n\tstart relative to contig: "+str(position_of["Cas9"][0])+"\t| start relative to locus: "+str(position_of["Cas9"][0]-min_locus)+" \n\t stop relative to contig: "+str(position_of["Cas9"][1])+"\t|  stop relative to locus: "+str(position_of["Cas9"][1]-min_locus)+" \n\t  length (nt): "+str(position_of["Cas9"][1]-position_of["Cas9"][0])+" \n\t  length (aa): "+str(int((position_of["Cas9"][1]-position_of["Cas9"][0])/3))+"\n"+" \n\t       strand: "+feature_strand
cas9_seqs=" \n\t\t\t\t --> SEQUENCE (aa) <-- \n"+textwrap.fill(Cas9_fasta_header,80)+"\n"+textwrap.fill(str(cas9_aa),80)+" \n\n\t\t\t\t --> SEQUENCE (nt) <-- \n"+textwrap.fill(Cas9_fasta_header,80)+"\n"+textwrap.fill(str(coso_non_capisco_piu_nulla[".ffn"]["Cas9"]),80)
print(cas1_out+cas2_out+cas9_out)

  | Cas1 |        	prokka ID:CJFLGLGO_01366
	start relative to contig: 45129	| start relative to locus: 643 
	 stop relative to contig: 46028	|  stop relative to locus: 1542 
	  length (nt): 899 
	  length (aa): 299
  | Cas2 |        	prokka ID:CJFLGLGO_01365
	start relative to contig: 44816	| start relative to locus: 330 
	 stop relative to contig: 45136	|  stop relative to locus: 650 
	  length (nt): 320 
	  length (aa): 106
  | Cas9 |        	prokka ID:CJFLGLGO_01367
	start relative to contig: 46031	| start relative to locus: 1545 
	 stop relative to contig: 49270	|  stop relative to locus: 4784 
	  length (nt): 3239 
	  length (aa): 1079
 
	       strand: -1


In [137]:
crispr_out="| CRISPRarray |        \n\t\tstart relative to contig: "+str(position_of["CRISPR"][0])+"\t| start relative to locus: "+str(position_of["CRISPR"][0]-min_locus)+" \n\t\t stop relative to contig: "+str(position_of["CRISPR"][1])+"\t|  stop relative to locus: "+str(position_of["CRISPR"][1]-min_locus)+" \n\t  length (nt): "+str(position_of["CRISPR"][1]-position_of["CRISPR"][0])+" \n\t       strand: "+str(strand)+ "\n\t    # repeats: "


In [138]:
mincedoutvero="""CRISPR 1   Range: 44486 - 44718
POSITION	REPEAT				SPACER
--------	------------------------------------	-----------------------------
44486		ATTATAGTTTCTTAATTTTTCTTGGTATGTTATAAT	AAGGTAATGGTCTTACCGAAGCTAATCTT	[ 36, 29 ]
44551		ATTGTAGTTCCCTAATTTTTCTTGGTATATTATAAT	CCGGCACCAACCGACTCGTCACTCATGACC	[ 36, 30 ]
44617		ATTGTAGTTCCCTAATTTTTCTTGGTATGTTATAAT	GAATTAAGATTCAAAGGGTCAGTAGGAAGA	[ 36, 30 ]
44683		ATTGTAGTTCCCTAATTTTTCTTGGTATGTTATAAT	
--------	------------------------------------	-----------------------------
Repeats: 4	Average Length: 36		Average Length: 29"""
print(mincedoutvero)

CRISPR 1   Range: 44486 - 44718
POSITION	REPEAT				SPACER
--------	------------------------------------	-----------------------------
44486		ATTATAGTTTCTTAATTTTTCTTGGTATGTTATAAT	AAGGTAATGGTCTTACCGAAGCTAATCTT	[ 36, 29 ]
44551		ATTGTAGTTCCCTAATTTTTCTTGGTATATTATAAT	CCGGCACCAACCGACTCGTCACTCATGACC	[ 36, 30 ]
44617		ATTGTAGTTCCCTAATTTTTCTTGGTATGTTATAAT	GAATTAAGATTCAAAGGGTCAGTAGGAAGA	[ 36, 30 ]
44683		ATTGTAGTTCCCTAATTTTTCTTGGTATGTTATAAT	
--------	------------------------------------	-----------------------------
Repeats: 4	Average Length: 36		Average Length: 29


In [139]:
#tracr
tracr_out="  | tracrRNA |        \n\t\tstart relative to contig: "+str(position_of["tracrRNA"][0])+"\t| start relative to locus: "+str(position_of["tracrRNA"][0]-min_locus)+" \n\t\t stop relative to contig: "+str(position_of["tracrRNA"][1])+"\t|  stop relative to locus: "+str(position_of["tracrRNA"][1]-min_locus)+" \n\t  length (nt): "+str(position_of["tracrRNA"][1]-position_of["tracrRNA"][0])+" \n\t       strand: "+str(strand)#      +" \n\t\t\t\t --> SEQUENCE <-- \n"+textwrap.fill(str(tracrRNA),80)

In [140]:
contigout="\n"+" "*int((80-len("CONTIG: "+contigname))/2)+"CONTIG: "+contigname+" "*int((80-len("CONTIG: "+contigname))/2)+"\n"\
+" "*int((80-len("GENOME: "+genomename))/2)+"GENOME: "+genomename+" "*int((80-len("GENOME: "+genomename))/2)\
+"\n\npositions are 0-based\n\n"
print(contigout)


                   CONTIG: NODE_137_length_63930_cov_6.02239                   
                    GENOME: Obregon-TitoAJ_2015__SM40__bin.4                    

positions are 0-based




In [141]:
print("writing locus positions file...")
f=open(outputdir+seqid+"_info.txt","w")
f.write(header+title+contigout+Cas_header+cas1_out+cas2_out+cas9_out+crispr_header+crispr_out+tracr_header+tracr_out)
f.close()
print("Done.")
print(header+title+contigout+Cas_header+cas1_out+cas2_out+cas9_out+crispr_header+crispr_out+tracr_header+tracr_out)

writing locus positions file...
Done.
% created on 04/11/2019
% author: L-F-S
% affiliation: University of Trento, Italy
% e-mail: lorenzo.signorini@alumni.unitn.it
% source code available at: https://github.com/L-F-S/cas_mining



                              CJFLGLGO_01367 LOCUS                              

                   CONTIG: NODE_137_length_63930_cov_6.02239                   
                    GENOME: Obregon-TitoAJ_2015__SM40__bin.4                    

positions are 0-based





				+-------------+
				 Cas Proteins
				+-------------+
  | Cas1 |        	prokka ID:CJFLGLGO_01366
	start relative to contig: 45129	| start relative to locus: 643 
	 stop relative to contig: 46028	|  stop relative to locus: 1542 
	  length (nt): 899 
	  length (aa): 299
  | Cas2 |        	prokka ID:CJFLGLGO_01365
	start relative to contig: 44816	| start relative to locus: 330 
	 stop relative to contig: 45136	|  stop relative to locus: 650 
	  length (nt): 320 
	  length (aa): 106
  | Cas9

In [142]:
metadata_title="\n"+"="*80+"\n\n"+" "*int((80-len(seqid+" metadata"))/2)+seqid+" metadata"+" "*int((80-len(seqid+" metadata"))/2)+"\n\n"+"="*80
m1="\nContig: \n\t"+contigname+"\nGenome: \n\t"+genomename+"\nSpecies (SGB): \n\t"+str(SGB)+"\nUnknown SGB?\n\t"+uSGB+"\nEstimated taxonomy: \n\t"+textwrap.fill(est_tax,80)+"\nLevel of estmated taxonomy\n\t"+l_est_tax


In [143]:
# save metadata file
print("writing metadata file...")
f=open(outputdir+seqid+"_metadata.txt","w")
f.write(header+metadata_title+m1)
f.close()
print("Done.")
print(header+title+contigout+Cas_header+cas1_out+cas2_out+cas9_out+crispr_header+crispr_out+tracr_header+tracr_out)

writing metadata file...
Done.
% created on 04/11/2019
% author: L-F-S
% affiliation: University of Trento, Italy
% e-mail: lorenzo.signorini@alumni.unitn.it
% source code available at: https://github.com/L-F-S/cas_mining



                              CJFLGLGO_01367 LOCUS                              

                   CONTIG: NODE_137_length_63930_cov_6.02239                   
                    GENOME: Obregon-TitoAJ_2015__SM40__bin.4                    

positions are 0-based





				+-------------+
				 Cas Proteins
				+-------------+
  | Cas1 |        	prokka ID:CJFLGLGO_01366
	start relative to contig: 45129	| start relative to locus: 643 
	 stop relative to contig: 46028	|  stop relative to locus: 1542 
	  length (nt): 899 
	  length (aa): 299
  | Cas2 |        	prokka ID:CJFLGLGO_01365
	start relative to contig: 44816	| start relative to locus: 330 
	 stop relative to contig: 45136	|  stop relative to locus: 650 
	  length (nt): 320 
	  length (aa): 106
  | Cas9 |     

In [119]:
# keep this for v2....
min_locus=min(value[0] for value in position_of.values())
max_locus=max(value[1] for value in position_of.values())
scaling_factor=(max_locus-min_locus)/70  # to have a locus of total length 70
scaled_pos_of={key:((value[0]-min_locus)/scaling_factor,(value[1]-min_locus)/scaling_factor) for (key, value) in position_of.items()}
print((scaled_pos_of))

{'CRISPR': (0.0, 9.101964452759589), 'Cas9': (25.43311506080449, 67.65575304022451), 'Cas1': (13.842843779232927, 25.38072965388213), 'Cas2': (9.743685687558465, 13.77736202057998), 'tracrRNA': (170.75023386342374, 70.0)}


In [128]:
print("copying genome and genome annotation..")
gff="/scratchCM/tmp_projects/epasolli_darkmatter/allcontigs/"+dataset+"/metabat/genomes_comp50_cont05/prokka/"+genomename+"/"+genomename+".gff"
system("cp "+genomefilename+" "+outputdir)
system("cp "+gff+" "+outputdir)

copying genome and genome annotation..


0